## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:  

Visualizar o impacto do peso, altura, idade, país e histórico dos atletas nos resultados das Olimpíadas de Verão ao longo dos anos, bem como, com esses dados, predizer os vencedores dos Jogos Olímpicos Tóquio 2020 de um dado esporte.

+ Assess situation:  


+ Determine data mining goals:  

Coletar dados sobre os atletas e confederações, dentre os dados que pretendemos utilizar estão idade, peso, altura, NOC, eventos nos quais participou, ano, quantidade de medalhas e a medalha obtida na edição em questão, que é nossa variável _target_.

+ Produce project plan:  
    + Coletar os dados dos medalhistas e quadro de medalhas da olimpíada de 2021
    + Criar uma base de dados SQLite e normalizar os dados
    + Gerar visualizações e explorar os dados do dataset gerado
    + Utilizar regressões para 



## II. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
Dados a serem calculados: (apenas um rascunho para o trabalho futuro)
    + quantidade de medalhas de cada país
    + quantidade de medalhas para cada atleta
    + quantas olimpíadas que cada atleta participou
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 